This notebook is mostly initial analysis and me just exploring the data.
For all visualisations and further analysis, see analysis 2.

In [99]:
# Import data manipulation libraries
import numpy as np 
import pandas as pd 

In [98]:
df = pd.read_excel("Candidate Assignment - Data Analyst 8.18.22.xlsx",sheet_name='Data')
df.head()

,Description,Market,Date,Network,Households,Hours
0,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,WHAM - ABC,11090.0,4554.0
1,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,WHEC - NBC,8037.0,3242.0
2,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,SPECTRUM NEWS - ROCHESTER,7157.0,2601.0
3,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,WROC - CBS,3635.0,1413.0
4,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,FOX NEWS CHANNEL,3216.0,1257.0


In [100]:
 # Just get a general idea of the data, what the 'descriptions' key is/whether it matters,
 # how many networks i'm looking at, if i need to exclude certain markets, whats the time frame, etc.. 
 
 descriptions = df['Description'].unique() #what diff descriptions are  - idk why these exist but maybe they are important for eval?
 markets = df['Market'].unique()   #how many markets are there? - only 1 - rochester
 networks = df['Network'].unique()  # how many different networks are we analyzing - 7
 len(networks) # -
dates = df['Date'].unique() # how many dates are there - 66, this is a 3 month span, exluding weekends 
len(dates)



66

In [101]:
# Add a column called Avg hours per household - divides daily households by hours watched, converts to minutes - this will increase readability of data
df['AVG_Min_per_Household'] = (df['Hours']/df['Households']) * 60
df.head()

,Description,Market,Date,Network,Households,Hours,AVG_Min_per_Household
0,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,WHAM - ABC,11090.0,4554.0,24.638413
1,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,WHEC - NBC,8037.0,3242.0,24.203061
2,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,SPECTRUM NEWS - ROCHESTER,7157.0,2601.0,21.805226
3,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,WROC - CBS,3635.0,1413.0,23.323246
4,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,FOX NEWS CHANNEL,3216.0,1257.0,23.451493


In [133]:
# Get a general sense of how spectrum is doing compared to other netowrks
quicksumm = df.groupby('Network').agg({'Households':np.mean, 'Hours': np.mean, 'AVG_Min_per_Household':np.mean}).sort_values('Households', ascending=False).reset_index()
quicksumm

# Quickly visualize it
import altair as alt
alt.Chart(quicksumm).mark_bar().encode(
    x='Network',
    y = 'Hours',
    color = 'Network'

)

alt.Chart(...)

In [110]:
# Split into before and after program launch date
# Assign new column called - before program

# launch date = july 17th

df['Date'].apply(pd.to_datetime)
beforeLaunchdf = df.set_index('Date')[:'2018-07-16']
afterLaunchDf = df.set_index('Date')['2018-07-17':]

# Sanity check:
print(len(df) == (len(beforeLaunchdf)+len(afterLaunchDf)))
afterLaunchDf.head(100)

True


,Description,Market,Network,Households,Hours,AVG_Min_per_Household
Date,,,,,,
2018-07-17,Analyst Test 63000-70000,Rochester-Ny,WHAM - ABC,9518.0,3856.0,24.307628
2018-07-17,Analyst Test 63000-70000,Rochester-Ny,WHEC - NBC,7171.0,2865.0,23.971552
2018-07-17,Analyst Test 63000-70000,Rochester-Ny,SPECTRUM NEWS - ROCHESTER,6378.0,2273.0,21.382879
2018-07-17,Analyst Test 63000-70000,Rochester-Ny,WROC - CBS,3395.0,1301.0,22.992636
2018-07-17,Analyst Test 63000-70000,Rochester-Ny,FOX NEWS CHANNEL,3008.0,1163.0,23.198138
...,...,...,...,...,...,...
2018-08-03,Analyst Test 63000-70000,Rochester-Ny,FOX NEWS CHANNEL,2612.0,1006.0,23.108729
2018-08-03,Analyst Test 63000-70000,Rochester-Ny,MSNBC,2034.0,807.0,23.805310
2018-08-03,Analyst Test 63000-70000,Rochester-Ny,CNN,1507.0,541.0,21.539482


In [244]:
# Get a general idea for what the new program is doing to our numbers at spectrum 


beforeSeries = beforeLaunchdf.groupby('Network').agg({'Households':np.mean, 'Hours': np.mean, 'AVG_Min_per_Household':np.mean}).sort_values('Households', ascending=False)
afterSeries = afterLaunchDf.groupby('Network').agg({'Households':np.mean, 'Hours': np.mean, 'AVG_Min_per_Household':np.mean}).sort_values('Households', ascending=False)
newdf = pd.concat([beforeSeries.loc['SPECTRUM NEWS - ROCHESTER'],afterSeries.loc['SPECTRUM NEWS - ROCHESTER']], axis=1)
newdf.columns = ['Before July 17', 'After July 17']
newdf['% Change'] = ((newdf['After July 17']/newdf['Before July 17'])-1) *100
print('Spectrum Data:')
newdf

Spectrum Data:


,Before July 17,After July 17,% Change
Households,8279.645833,7143.867647,-13.717715
Hours,3098.703125,2658.511438,-14.205675
AVG_Min_per_Household,22.452732,22.280212,-0.768369


In [123]:
# At first glance, it seems like the new program is causing us to lose viewers, as there were much less viewers in the weeks after the launch
# However, we have to consider other factors, we should look at other networks as well
# Whats going on at CNN?
competitordf = pd.concat([beforeSeries.loc['CNN'],afterSeries.loc['CNN']], axis=1)
competitordf.columns = ['Before Program', 'After Program']
competitordf['% Change'] = ((competitordf['After Program']/competitordf['Before Program'])-1) *100
print('CNN Data:')
competitordf

CNN Data:


,Before Program,After Program,% Change
Households,2306.604167,2140.576797,-7.197913
Hours,860.630208,799.611111,-7.090048
AVG_Min_per_Household,22.322983,22.305865,-0.076681


In [124]:
# We can copy paste to get a better idea of whats going on
# Whats going on at ABC?
competitordf = pd.concat([beforeSeries.loc['WHAM - ABC'],afterSeries.loc['WHAM - ABC']], axis=1)
competitordf.columns = ['Before Program', 'After Program']
competitordf['% Change'] = ((competitordf['After Program']/competitordf['Before Program'])-1) *100
print('ABC Data:')
competitordf

ABC Data:


,Before Program,After Program,% Change
Households,10534.572917,8851.401961,-15.977591
Hours,4462.125000,3736.941176,-16.251984
AVG_Min_per_Household,25.375446,25.280066,-0.375874


In [125]:
# At NBC?
competitordf = pd.concat([beforeSeries.loc['WHEC - NBC'],afterSeries.loc['WHEC - NBC']], axis=1)
competitordf.columns = ['Before Program', 'After Program']
competitordf['% Change'] = ((competitordf['After Program']/competitordf['Before Program'])-1) *100
print('NBC Data:')
competitordf

NBC Data:


,Before Program,After Program,% Change
Households,8813.489583,7510.053922,-14.789099
Hours,3716.291667,3156.924837,-15.051747
AVG_Min_per_Household,25.271441,25.179910,-0.362193


In [126]:
# At fox?
competitordf = pd.concat([beforeSeries.loc['FOX NEWS CHANNEL'],afterSeries.loc['FOX NEWS CHANNEL']], axis=1)
competitordf.columns = ['Before Program', 'After Program']
competitordf['% Change'] = ((competitordf['After Program']/competitordf['Before Program'])-1) *100
print('Fox Data:')
competitordf


Fox Data:


,Before Program,After Program,% Change
Households,4449.635417,3597.892157,-19.141866
Hours,1817.218750,1458.459150,-19.742235
AVG_Min_per_Household,24.450392,24.242242,-0.851317


In [132]:
# I just want to average change in general
avgAllNetworksdf = pd.concat([beforeSeries.mean(), afterSeries.mean()], axis=1)
avgAllNetworksdf.columns = ['Before Program', 'After Program']
avgAllNetworksdf['% Change'] = ((avgAllNetworksdf['After Program']/avgAllNetworksdf['Before Program'])-1) *100
print('All Netorks Averaged:')
avgAllNetworksdf

All Netorks Averaged:


,Before Program,After Program,% Change
Households,5918.233631,5072.507470,-14.290179
Hours,2401.045387,2048.978992,-14.663046
AVG_Min_per_Household,24.046214,23.939610,-0.443330


In [242]:
# I just want to average change in general
# Now i want to drop to spectrum to just have
avgAllNetworksdf = pd.concat([beforeSeries.drop('SPECTRUM NEWS - ROCHESTER').mean(), afterSeries.drop('SPECTRUM NEWS - ROCHESTER').mean()], axis=1)
avgAllNetworksdf.columns = ['Before July 17', 'After July 17']
avgAllNetworksdf['% Change'] = ((avgAllNetworksdf['After July 17']/avgAllNetworksdf['Before July 17'])-1) *100
print('All Netorks Averaged:')
avgAllNetworksdf

All Netorks Averaged:


,Before July 17,After July 17,% Change
Households,5524.664931,4727.280773,-14.433168
Hours,2284.769097,1947.390251,-14.766431
AVG_Min_per_Household,24.311795,24.216176,-0.393299


In [142]:
# To compare
pd.concat([avgAllNetworksdf['% Change'], newdf['% Change']], axis=1)

,% Change,% Change
Households,-14.433168,-13.717715
Hours,-14.766431,-14.205675
AVG_Min_per_Household,-0.393299,-0.768369


In [95]:
# Before vis:

alt.Chart(beforeSeries.reset_index()).mark_bar().encode(
    x='Network',
    y = 'Households',
    color = 'Network'

)

alt.Chart(afterSeries.reset_index()).mark_bar().encode(
    x='Network',
    y = 'Households',
    color = 'Network'

)


alt.Chart(...)

In [201]:
# Probably should just have done a time series analysis from the start

df = pd.read_excel("Candidate Assignment - Data Analyst 8.18.22.xlsx",sheet_name='Data')
df['AVG_Min_per_Household'] = (df['Hours']/df['Households']) * 60
df.head()



,Description,Market,Date,Network,Households,Hours,AVG_Min_per_Household
0,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,WHAM - ABC,11090.0,4554.0,24.638413
1,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,WHEC - NBC,8037.0,3242.0,24.203061
2,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,SPECTRUM NEWS - ROCHESTER,7157.0,2601.0,21.805226
3,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,WROC - CBS,3635.0,1413.0,23.323246
4,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,FOX NEWS CHANNEL,3216.0,1257.0,23.451493


In [192]:
# Boolean masking scratch 
df[(df['Description'] == 'Analyst Test 63000-70000') | (df['Description'] == 'Analyst Test 70000-73000')]

,Description,Market,Date,Network,Households,Hours,AVG_Min_per_Household
0,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,WHAM - ABC,11090.0,4554.0,24.638413
1,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,WHEC - NBC,8037.0,3242.0,24.203061
2,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,SPECTRUM NEWS - ROCHESTER,7157.0,2601.0,21.805226
3,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,WROC - CBS,3635.0,1413.0,23.323246
4,Analyst Test 63000-70000,Rochester-Ny,2018-06-01,FOX NEWS CHANNEL,3216.0,1257.0,23.451493
...,...,...,...,...,...,...,...
919,Analyst Test 70000-73000,Rochester-Ny,2018-08-31,SPECTRUM NEWS - ROCHESTER,6981.0,2513.0,21.598625
920,Analyst Test 70000-73000,Rochester-Ny,2018-08-31,WROC - CBS,3909.0,1551.0,23.806600
921,Analyst Test 70000-73000,Rochester-Ny,2018-08-31,FOX NEWS CHANNEL,3155.0,1262.0,24.000000
922,Analyst Test 70000-73000,Rochester-Ny,2018-08-31,MSNBC,2442.0,975.0,23.955774


In [218]:
# I want to isolate the show
# First get only spectrum 
spectrumdf = df[df['Network']=='SPECTRUM NEWS - ROCHESTER']
# Then i want to get only 7-9am
SevenToNinedf = spectrumdf[(df['Description'] == 'Analyst Test 70000-73000') | (df['Description'] == 'Analyst Test 73000-80000') | (df['Description'] == 'Analyst Test 80000-83000')| (df['Description'] == 'Analyst Test 83000-90000')]
# Then I want to get after the launch date
showdf = SevenToNinedf.set_index('Date')['2018-07-17':]
preshowdf = SevenToNinedf.set_index('Date')[:'2018-07-16']
showdf

C:\Users\jackm\AppData\Local\Temp/ipykernel_15128/108773015.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  SevenToNinedf = spectrumdf[(df['Description'] == 'Analyst Test 70000-73000') | (df['Description'] == 'Analyst Test 73000-80000') | (df['Description'] == 'Analyst Test 80000-83000')| (df['Description'] == 'Analyst Test 83000-90000')]


,Description,Market,Network,Households,Hours,AVG_Min_per_Household
Date,,,,,,
2018-07-17,Analyst Test 70000-73000,Rochester-Ny,SPECTRUM NEWS - ROCHESTER,7134.0,2597.0,21.841884
2018-07-18,Analyst Test 70000-73000,Rochester-Ny,SPECTRUM NEWS - ROCHESTER,2294.0,839.0,21.944202
2018-07-19,Analyst Test 70000-73000,Rochester-Ny,SPECTRUM NEWS - ROCHESTER,6961.0,2516.0,21.686539
2018-07-20,Analyst Test 70000-73000,Rochester-Ny,SPECTRUM NEWS - ROCHESTER,6969.0,2589.0,22.290142
2018-07-23,Analyst Test 70000-73000,Rochester-Ny,SPECTRUM NEWS - ROCHESTER,7217.0,2601.0,21.623943
...,...,...,...,...,...,...
2018-08-27,Analyst Test 83000-90000,Rochester-Ny,SPECTRUM NEWS - ROCHESTER,7327.0,2744.0,22.470315
2018-08-28,Analyst Test 83000-90000,Rochester-Ny,SPECTRUM NEWS - ROCHESTER,8354.0,3392.0,24.361982
2018-08-29,Analyst Test 83000-90000,Rochester-Ny,SPECTRUM NEWS - ROCHESTER,7485.0,2810.0,22.525050


In [219]:
showdf.groupby('Description').agg({'Hours':np.mean,'Households':np.mean,'AVG_Min_per_Household':np.mean}).mean()

Hours                    2751.897059
Households               7388.107843
AVG_Min_per_Household      22.330179
dtype: float64

In [220]:
timeblockcomparison = pd.concat([(preshowdf.groupby('Description').agg({'Hours':np.mean,'Households':np.mean,'AVG_Min_per_Household':np.mean}).mean()), (showdf.groupby('Description').agg({'Hours':np.mean,'Households':np.mean,'AVG_Min_per_Household':np.mean}).mean())], axis=1)
timeblockcomparison.columns = ['Before Program', 'After Program']
timeblockcomparison['% Change'] = ((timeblockcomparison['After Program']/timeblockcomparison['Before Program'])-1) *100
timeblockcomparison
# an isolated show data:

,Before Program,After Program,% Change
Hours,3189.546875,2751.897059,-13.721379
Households,8515.890625,7388.107843,-13.243275
AVG_Min_per_Household,22.492115,22.330179,-0.719970


In [225]:
# In the next few cells i just want to get some general performace metrics about the show
# A few descriptive statistics to begin my show 
byday = showdf.groupby('Date').agg({'Households':[np.sum,np.mean], 'Hours': [np.mean, np.sum]})
byday.head()

Households                 Hours         
                     sum         mean     mean      sum
Date                                                   
2018-07-17  30211.000000  7552.750000  2803.25  11213.0
2018-07-18   9306.666667  2326.666667   869.75   3479.0
2018-07-19  29790.000000  7447.500000  2748.75  10995.0
2018-07-20  31932.000000  7983.000000  3026.50  12106.0
2018-07-23  30241.000000  7560.250000  2784.25  11137.0

In [227]:
# AVG Daily Hours 
byday['Hours']['sum'].mean()

11007.588235294117

In [228]:
#AVG Households by block
byday['Households']['mean'].mean()


7388.107843137254

In [241]:
# Household converted to people
round(byday['Households']['mean'].mean() * 2.65)

19578

In [238]:
# Penetration from subscriber base

str(round(((byday['Households']['mean'].mean() / 250000) * 100),2)) + ' %'

'2.96 %'